# Artificial Neural Network


## Importing Libraries


In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [23]:
tf.__version__

'2.15.0'

## Part 1: Data Preprocessing


### Importing the dataset


In [24]:
dataset = pd.read_csv("../datasets/12_bank_customer_churn_prediction.csv")
X = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values

In [25]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [26]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data


#### Label Encoding the "Gender" column


In [27]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [28]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


#### One Hot Encoding the "Geography" column


In [29]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(
    transformers=[("encoder", OneHotEncoder(), [1])], remainder="passthrough"
)
X = np.array(ct.fit_transform(X))

In [50]:
print(X[6])

[1.0 0.0 0.0 822 1 50 7 0.0 2 1 1 10062.8]


### Splitting the dataset into Training and Test Sets


In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Feature Scaling


In [42]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

## Part 2: Building the ANN


### Initializing the ANN


In [43]:
ann = tf.keras.models.Sequential()  # initializes our ann as a sequence of layers

### Adding the input layer and the first hidden layer


In [44]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))
# units is number of neurons on the first hidden layer
# relu refers to the rectifier activation function

### Adding the second hidden layer


In [45]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

### Adding the output layer


In [46]:
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

## Part 3: Training the ANN


### Compiling the ANN


In [47]:
ann.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)  # for non-binary the loss is categorical_crossentropy

### Training the ANN on the Training set


In [48]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 845us/step - loss: 0.5734 - accuracy: 0.7423
Epoch 2/100
250/250 [==============================] - 0s 836us/step - loss: 0.4719 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 867us/step - loss: 0.4455 - accuracy: 0.8016
Epoch 4/100
250/250 [==============================] - 0s 924us/step - loss: 0.4308 - accuracy: 0.8110
Epoch 5/100
250/250 [==============================] - 0s 931us/step - loss: 0.4201 - accuracy: 0.8170
Epoch 6/100
250/250 [==============================] - 0s 927us/step - loss: 0.4116 - accuracy: 0.8174
Epoch 7/100
250/250 [==============================] - 0s 861us/step - loss: 0.4044 - accuracy: 0.8206
Epoch 8/100
250/250 [==============================] - 0s 907us/step - loss: 0.3984 - accuracy: 0.8221
Epoch 9/100
250/250 [==============================] - 0s 937us/step - loss: 0.3935 - accuracy: 0.8242
Epoch 10/100
250/250 [==============================] - 0s 911us/step - l

## Part 4: Making the prediciton and evaluating the model


### Predicting the result of a single observation


**Homework**
Use or ANN model to predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40 years old
Tenure: 3 years
Balance: $60000
Number of Products: 2
Does this custome have a credit card? Yes
Is this customer an Active Member: Yes
Estimated Salary: $50000
So should we say goodbye to the customer?

**Solution**


In [57]:
print(
    ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5
)

1/1 [==============================] - 0s 17ms/step
[[False]]


Therefore our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the 'predict' method always expects a 2D array as the format of the inputs. And puttingour values into a double square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that 'France' country was not input as a string in the last column but as '1,0,0' in the first three columns. That's because of course the predict method expects the one-hot encoded values of the state, and as we see in the first row of the matrix of features X, France was encoded as '1,0,0.' And be careful to include these values in the first three columns beacuse the dummy variables are always in the first columns.


### Prediciting the test results


In [58]:
y_pred = ann.predict(X_test)
y_pred = y_pred > 0.5

print(
    np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1)
)

63/63 [==============================] - 0s 690us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix


In [59]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1538   57]
 [ 218  187]]


0.8625